# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import api_key
from configs import g_keys

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Configure gmaps
gmaps.configure(api_key=g_keys)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [32]:
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Find median coordinates
mid_lat = weather_data["Lat"].median()
mid_long = weather_data["Lng"].median()
center = [mid_lat, mid_long]

#Create Figure
fig = gmaps.figure(zoom_level=1, center = center)

# Add heatmap layer to figure
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
# Narrow down the cities with wind speed less than 35 mph, cloudiness equals to 0 and max temp between 60 and 75
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 35) & (weather_data["Cloudiness"] == 20) & \
                                   (weather_data["Max Temp"] >= 60) & (weather_data["Max Temp"] <= 75)].dropna()

narrowed_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
68,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99
69,69,fortuna,20,ES,1558378767,77,38.18,-1.13,69.80,4.70
78,78,crotone,20,IT,1558378768,67,39.09,17.12,62.60,10.29
86,86,nikolskoye,20,RU,1558378770,56,59.70,30.79,69.01,4.47
117,117,east london,20,ZA,1558378775,72,-33.02,27.91,60.80,27.51
160,160,trapani,20,IT,1558378785,72,38.02,12.54,66.20,17.22
188,188,lagoa,20,PT,1558378790,46,37.14,-8.45,73.40,18.34
223,223,koronowo,20,PL,1558378736,60,53.31,17.94,71.01,3.36
303,303,yulara,20,AU,1558378845,77,-25.24,130.99,62.60,8.16
360,360,fort nelson,20,CA,1558378858,42,58.81,-122.69,68.00,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = "Hyatt"

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
68,beja,PT,38.02,-7.87,Hyatt
69,fortuna,ES,38.18,-1.13,Hyatt
78,crotone,IT,39.09,17.12,Hyatt
86,nikolskoye,RU,59.70,30.79,Hyatt
117,east london,ZA,-33.02,27.91,Hyatt
160,trapani,IT,38.02,12.54,Hyatt
188,lagoa,PT,37.14,-8.45,Hyatt
223,koronowo,PL,53.31,17.94,Hyatt
303,yulara,AU,-25.24,130.99,Hyatt
360,fort nelson,CA,58.81,-122.69,Hyatt


In [35]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_keys}

In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
print(locations)

       Lat     Lng
68   38.02   -7.87
69   38.18   -1.13
78   39.09   17.12
86   59.70   30.79
117 -33.02   27.91
160  38.02   12.54
188  37.14   -8.45
223  53.31   17.94
303 -25.24  130.99
360  58.81 -122.69
374  38.42   -7.53
522  61.99    5.14


In [9]:
#Format map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…